# 사용자의 평가경향을 고려한 CF

In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

base_src = '../data/drive-download-20240102T142504Z-001'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    u_user_src,
    sep='|',
    names=u_cols,
    encoding='latin-1'
)
users = users.set_index('user_id')

u_item_src = os.path.join(base_src, 'u.item')
i_cols = [
    'movie_id', 'title', 'release date', 'video release date',
    'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animat ion',
    'Children\'s', 'Comedy', 'Crime', 'Documentary ', 'Drama',
    'Fantasy', 'Film- Noir', 'Horror', 'Musical', 'Mystery',
    'Romance ', 'Sci-Fi', 'Thriller', 'War', 'Western'
]
movies = pd.read_csv(
    u_item_src,
    sep='|',
    names=i_cols,
    encoding='latin-1'
)
movies = movies.set_index('movie_id')

u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=r_cols,
    encoding='latin-1'
)


# 정확도 (RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))


# 모델별 RMSE를 계산하는 함수
def score(model, neighbor_size=0):
    # 테스트 데이터의 user_id와 movie)id 간 pair를 맞춰 튜플형 원소 리스트 데이터 생성
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    # 모든 사용자 - 영화 pair에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    # 실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# 데이터셋 생성
x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, stratify=y
)

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# 코사인 유사도를 구하기 위해 rating값을 복사하고, 계산 시 NaN값 에러 대비 0으로 변환
matrix_dummy = rating_matrix.copy().fillna(0)
# 모든 사용자간 코사인 유사고 계산
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
# 필요한 값 조회를 위해 인덱스 및 컬럼명 지정
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)


In [7]:
# 사용자 펴악 경향을 고려한 함수
# full matrix에서 각 사용자의 평점평균을 구한다.
rating_mean = rating_matrix.mean(axis=1)
# 사용자 평점 평균의 편차 구하기
rating_bias = (rating_matrix.T - rating_mean).T

# 사용자 평가 경향을 고려한 함수
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현재 영화에 대한 모든 사용자의 평점편차를 구한다.
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_bias[movie_id].copy() 
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        # Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:
            # 현재 영화에 대한 모든 사용자의 평점편차를 이용해 예측값을 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 평점편차에 사용자의 평균을 더한다.
            prediction = prediction + rating_mean[user_id]
        # Neighbor size가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:] 
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 평점편차에 사용자의 평균을 더한다.
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [8]:
# neighbor sizer가 10,20,30,40,50,60 인 경우에 대해서 RMSE를 계산하고 이를 출력 
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print('Neighbor size = %d : RMSE = %.4f' % (neighbor_size, score(CF_knn_bias, neighbor_size)))
 

Neighbor size = 10 : RMSE = 0.9594
Neighbor size = 20 : RMSE = 0.9500
Neighbor size = 30 : RMSE = 0.9489
Neighbor size = 40 : RMSE = 0.9492
Neighbor size = 50 : RMSE = 0.9499
Neighbor size = 60 : RMSE = 0.9507
